# Chapter 0: Compiling and Running Toy

Here is a simple program in the Toy programming language running in a RISC-V emulator, 
compiled using xDSL.
Try changing the program and observing the output:

In [1]:
from xdsl.utils.exceptions import VerifyException

from toy.compiler import compile, emulate_riscv

program = """
def main() {
  # Define a variable `a` with shape <2, 3>, initialized with the literal value.
  # The shape is inferred from the supplied literal.
  var a = [[1, 2, 3], [4, 5, 6]];

  # b is identical to a, the literal tensor is implicitly reshaped: defining new
  # variables is the way to reshape tensors (element count must match).
  var b<3, 2> = [1, 2, 3, 4, 5, 6];

  # There is a built-in print instruction to display the contents of the tensor
  print(b);

  # Reshapes are implicit on assignment
  var c<2, 3> = b;

  # There are + and * operators for pointwise addition and multiplication
  var d = a + c;

  print(d);
}
"""

try:
  code = compile(program)
  emulate_riscv(code)
except VerifyException as e:
  print(e)

[CPU] Started running from example.asm:.text at heap (0x100) + 0x46c
Program(name=example.asm,sections=set(),base=['.bss', '.data', '.text'])
[[1, 2], [3, 4], [5, 6]]
[[2, 4, 6], [8, 10, 12]]
[CPU] Program exited with code 0
